**Image transformation**

This script transforms a set of images according to the previously trained model.

**Important:** the images to transform need to be located in a folder named *original_images* in the root, and will be output in a folder named *transformed_images* in the root as well.

In [ ]:
import torch, torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch import nn, optim
from torchvision.utils import make_grid
from torchvision.utils import save_image
from tqdm.notebook import tqdm
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from PIL import Image

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

class ImageData(Dataset):
    def __init__(self, root, transform=None):
        self.transform = transform
        self.frame = []
        for file in os.listdir(root):
            f = os.path.join(root, file)
            if os.path.isfile(f):
                self.frame.append(f)
    
    def __len__(self):
        return len(self.frame)
    
    
    def __getitem__(self, index):
        image_name = self.frame[index]
        image = Image.open(image_name)
        if self.transform:
            image = self.transform(image)
        return image

rootOrig = './original_images'
rootTran = './transformed_images'
origset = ImageData(root=rootOrig, transform=transform)

if torch.cuda.is_available():
    device=torch.device('cuda')
    print('Cuda!')
else:
    device=torch.device('cpu')
    print('CPU :(')

In [ ]:
class Filter1NN(nn.Module):
    def __init__(self,inchannels):
        super(Filter1NN,self).__init__()
        
        self.conv1 = nn.ConvTranspose2d(in_channels=inchannels, 
                               out_channels=3, 
                               kernel_size=3, 
                               stride=1, 
                               padding=1)
        self.lrelu1 = nn.Tanh()
        self.conv2 = nn.ConvTranspose2d(in_channels=3, 
                               out_channels=3, 
                               kernel_size=3, 
                               stride=1, 
                               padding=1)
        self.lrelu2 = nn.Tanh()
        self.conv4 = nn.ConvTranspose2d(in_channels=3, 
                               out_channels=3, 
                               kernel_size=3, 
                               stride=1, 
                               padding=1)
        self.sigm = nn.Sigmoid()

    def forward(self,x):
        x = self.conv1(x)
        x = self.lrelu1(x)
        x = self.conv2(x )
        x = self.lrelu2(x)
        x = self.conv4(x)
        return self.sigm(x)
        
# F=Filter1NN(3).to(device)

In [ ]:
F = torch.load("./modelbest.pth")
F.eval()
outdir = "transformed_images"

isExist = os.path.exists(outdir)
if not isExist:
    os.makedirs(outdir)

for i in tqdm(range(len(origset))):
    im_o = origset[i].to(device).reshape(1,origset[i].shape[0],origset[i].shape[1],origset[i].shape[2])
    im_t = F(im_o)
    name_t = origset.frame[i].replace("input", "output")
    save_image(im_t,os.path.join(outdir, name_t.replace("./original_images\\","")))
    
    
    